In [1]:
import pymorphy2
import re
from collections import defaultdict
import telebot
from math import floor


In [2]:
class Question_analizier:
    morph = pymorphy2.MorphAnalyzer ()
    questions = []
    all_words = defaultdict(lambda: 0)
    
    def printf (self):
        print ('questions\n', self.questions, '\n')
    
    def get_set (self, question):
        words = re.findall(r'[^\s!,.?":;0-9]+', question)
        ret_set = set ()
        
        for word in words:
            self.all_words [word] += 1
            ret_set.add (self.morph.parse (word)[0].normal_form)
            
        return ret_set
    
    def add_question (self, question):
        self.questions.append (set())
        self.questions [-1] = self.get_set (question)
        
    def expend_question (self, question_id, another_question):
        set_to_add = self.get_set (another_question)
        self.questions [question_id] = self.questions [question_id] | set_to_add
        
    def get_closest_question (self, question):
        words = self.get_set (question)
        least_distance = float ("inf")
        least_id = -1
        
        for i in range (len(self.questions)):
            words_in_common = len (words & self.questions [i]) + 1 #not to devide by 0
            words_at_all = len (words | self.questions [i]) 
            
            if words_at_all/(words_in_common) < least_distance:
                least_distance = words_at_all/(words_in_common*words_in_common)
                least_id = i
                
        return least_id, least_distance
    
    def get_popular_themes (self, number):
        return sorted(self.all_words.items(), key=lambda item:item[1])[-number:]

In [3]:
class Question_interpriter:
    edited_questions = Question_analizier ()
    questions = []
    answers = []
    
    def add (self, question, answer):
        self.edited_questions.add_question (question + ' ' + answer)
        self.questions.append (question)
        self.answers.append (answer)
        
    def get_answer (self, question):
        least_id, least_distance = self.edited_questions.get_closest_question (question)
        
        if least_distance > 0: #should add some value, not 0
            return self.answers [least_id], least_id
        else:
            return 'may be this will help you\n' + 'question:\n' + questions [least_id] + '\nanswer:\n' + answers [least_id]
            
    def expend_question (self, question_id, question):
        self.edited_questions.expend_question (question_id, question)
        
    def printf (self):
        print ('questions\n')
        print (self.questions)
        print ('\nanswers\n')
        print (self.answers)
        self.edited_questions.printf ()
    
    def __init__ (self, path):
        counter = 0
        file = open (path, 'r')
        
        for line in file:
            if not line.isspace():
                
                if counter % 2 == 0:
                    self.edited_questions.add_question (line)
                    self.questions.append (line)
                else:                    
                    self.edited_questions.expend_question(floor (counter/2), line)
                    self.answers.append (line)
                
                counter += 1
        
        file.close
            

In [4]:
GoToHelp = Question_interpriter ('cases.txt')

In [5]:
token = '469148892:AAFTem8u1mydH0SXjPRTU8RmpU4R15lVSWc'
password = '2718281828459045'
bot = telebot.TeleBot (token)
admins_id = set ()
command_set = ''
expend_question = ''
expend_question_id = -1

In [6]:
@bot.message_handler (commands = ['start'])
def handle_start (message):
    user_markup = telebot.types.ReplyKeyboardMarkup (True)
    user_markup.row ('/ask_question', '/become_admin', '/add_case')
    bot.send_message (message.from_user.id, 'Выберите команду', reply_markup = user_markup)

@bot.message_handler (commands = ['add_case'])
def handle_add_case (message):
    global command_set
    global admins_id
    
    if message.from_user.id in admins_id:
        command_set = 'add_case'
        bot.send_message (message.from_user.id, 'Введите Вопрос и на следующей строке ответ')
    else:
        bot.send_message (message.from_user.id, 'У вас нет прав добавлять вопрос')

@bot.message_handler (commands = ['become_admin'])
def handle_become_admin (message):
    global command_set
    
    bot.send_message (message.from_user.id, 'Введите пароль')
    command_set = 'become_admin'
    
@bot.message_handler (commands = ['ask_question'])
def handle_ask_question (message):
    global command_set
    
    bot.send_message (message.from_user.id, 'Введите вопрос')
    command_set = 'ask_question'

@bot.message_handler (commands = ['yes'])
def handle_yes (message):
    global command_set
    
    bot.send_message (message.from_user.id, 'Рад, что смог вам помочь')
    command_set = 'yes'
    
    user_markup = telebot.types.ReplyKeyboardMarkup (True)
    user_markup.row ('/ask_question', '/become_admin', '/add_case')
    bot.send_message (message.from_user.id, 'Выберите команду', reply_markup = user_markup)
    
    
@bot.message_handler (commands = ['no'])
def handle_no (message):
    global command_set
    
    bot.send_message (message.from_user.id, ':С Мне жаль, что не смог вам помочь')
    
    user_markup = telebot.types.ReplyKeyboardMarkup (True)
    user_markup.row ('/ask_question', '/become_admin', '/add_case')
    bot.send_message (message.from_user.id, 'Выберите команду', reply_markup = user_markup)
    


In [7]:
@bot.message_handler (content_types = ['text'])
def handle_text (message):
    global command_set
    global admins_id
    global GoToHelp
    global expend_question
    global expend_question_id

    if command_set == '':
        bot.send_message (message.from_user.id, 'Выберите Команду')
        
    elif command_set == 'become_admin':
        if message.text == password:
            admins_id.add (message.from_user.id)
            bot.send_message (message.from_user.id, 'Верный пароль')
        else:
            bot.send_message (message.from_user.id, 'Неверный пароль')
                
    elif command_set == 'ask_question': 
        expend_question = message.text
        answer, expend_question_id = GoToHelp.get_answer (message.text)
        bot.send_message (message.from_user.id, answer)
        
        hide_markup = telebot.types.ReplyKeyboardRemove ()
        bot.send_message (message.from_user.id, 'Вы получили ответ на свой вопрос?', reply_markup = hide_markup)
        
        user_markup = telebot.types.ReplyKeyboardMarkup (True, True)
        user_markup.row ('/yes', '/no')
        bot.send_message (message.from_user.id, 'Choose wisely', reply_markup = user_markup)
                
    elif command_set == 'yes':
        GoToHelp.expend_question (expend_question_id, expend_question)
        
    elif command_set == 'add_case':
        GoToHelp.add (message.text.splitlines()[0], message.text.splitlines()[1])
        
    
    command_set = ''

        
        
        

In [ ]:
bot.polling(none_stop=True) 